# SciTeX Statistics Module - Comprehensive Tutorial

This notebook demonstrates the complete functionality of the `scitex.stats` module for statistical analysis and hypothesis testing.

## Features Covered
* Basic statistical tests (t-tests, ANOVA, chi-square)
* Correlation analysis with multiple comparison corrections
* Advanced hypothesis testing and bootstrap methods
* PyTorch tensor support for statistical operations
* Power analysis and effect size calculations
* Outlier detection and robust statistics
* Publication-ready statistical reporting

## Table of Contents
1. [Basic Statistical Tests](#1-basic-statistical-tests)
2. [Correlation Analysis](#2-correlation-analysis)
3. [ANOVA and Post-hoc Tests](#3-anova-and-post-hoc-tests)
4. [Bootstrap Methods](#4-bootstrap-methods)
5. [PyTorch Integration](#5-pytorch-integration)
6. [Outlier Detection](#6-outlier-detection)
7. [Power Analysis](#7-power-analysis)
8. [Advanced Statistical Operations](#8-advanced-statistical-operations)
9. [Complete Statistical Analysis Pipeline](#9-complete-statistical-analysis-pipeline)

In [ ]:
# Detect notebook name for output directory
import os
from pathlib import Path

# Get notebook name (for papermill compatibility)
notebook_name = "11_scitex_stats"
if 'PAPERMILL_NOTEBOOK_NAME' in os.environ:
    notebook_name = Path(os.environ['PAPERMILL_NOTEBOOK_NAME']).stem


In [ ]:
# Import required libraries
import sys
sys.path.insert(0, '../src')
import scitex as stx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from scitex import stats

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)


In [ ]:
# Path compatibility helper
import os
from pathlib import Path

def ensure_output_dir(subdir: str, notebook_name: str = "11_scitex_stats"):
    """Ensure output directory exists with backward compatibility."""
    expected_dir = Path(subdir)
    actual_dir = Path(f"{notebook_name}_out") / subdir
    
    if not expected_dir.exists() and actual_dir.exists():
        # Create symlink for backward compatibility
        try:
            os.symlink(str(actual_dir.resolve()), str(expected_dir))
        except (OSError, FileExistsError):
            pass
    
    return expected_dir


## 1. Basic Statistical Tests

The stats module provides comprehensive statistical testing functionality with automatic result formatting.

In [ ]:
import numpy as np
# Generate sample data for testing
group1 = np.random.normal(5, 2, 50)
group2 = np.random.normal(6, 2, 50)
group3 = np.random.normal(7, 2, 50)


In [ ]:
# Two-sample t-test
t_stat, p_value = stats.ttest_ind(group1, group2)

In [ ]:
# One-way ANOVA
f_stat, p_value = stats.f_oneway(group1, group2, group3)

In [ ]:
import numpy as np
# Chi-square test of independence
# Create contingency table
observed = np.array([[10, 10, 20], [20, 20, 40]])
chi2_stat, p_value = stats.chi2_contingency(observed)[:2]

In [ ]:
# Normality tests
from scipy import stats as scipy_stats

# Shapiro-Wilk test
shapiro_stat, shapiro_p = scipy_stats.shapiro(group1)

## 2. Correlation Analysis

Advanced correlation analysis with multiple comparison corrections and confidence intervals.

In [ ]:
import pandas as pd
import numpy as np
# Generate correlated data
n_samples = 100
x = np.random.randn(n_samples)
y = 0.7 * x + 0.3 * np.random.randn(n_samples)  # Correlation ~0.7
z = np.random.randn(n_samples)  # Independent variable

# Create DataFrame for analysis
df = pd.DataFrame({'x': x, 'y': y, 'z': z})


In [ ]:
# Pearson correlation with confidence intervals
r, p_val = scipy_stats.pearsonr(x, y)

# Spearman correlation (non-parametric)
rho, p_val_spear = scipy_stats.spearmanr(x, y)

In [ ]:
# Multiple correlation analysis with correction
from scipy.stats import pearsonr
from statsmodels.stats.multitest import multipletests

# Calculate all pairwise correlations
variables = ['x', 'y', 'z']
correlations = []
p_values = []
pairs = []

for i in range(len(variables)):
    for j in range(i+1, len(variables)):
        var1, var2 = variables[i], variables[j]
        r, p = pearsonr(df[var1], df[var2])
        correlations.append(r)
        p_values.append(p)
        pairs.append(f"{var1}-{var2}")

# Apply multiple comparison correction
corrected_p = multipletests(p_values, method='bonferroni')[1]

for pair, r, p_orig, p_corr in zip(pairs, correlations, p_values, corrected_p):
    pass  # Loop body

## 3. ANOVA and Post-hoc Tests

Comprehensive ANOVA analysis with post-hoc testing for multiple group comparisons.

In [ ]:
import pandas as pd
import numpy as np
# Create data for ANOVA
treatment_a = np.random.normal(10, 2, 30)
treatment_b = np.random.normal(12, 2, 30)
treatment_c = np.random.normal(14, 2, 30)
control = np.random.normal(8, 2, 30)

# Combine data for analysis
all_data = np.concatenate([treatment_a, treatment_b, treatment_c, control])
groups = (['A'] * 30 + ['B'] * 30 + ['C'] * 30 + ['Control'] * 30)

anova_df = pd.DataFrame({
    'value': all_data,
    'group': groups
})


In [ ]:
# One-way ANOVA
from scipy.stats import f_oneway

f_stat, p_value = f_oneway(treatment_a, treatment_b, treatment_c, control)

In [ ]:
# Post-hoc pairwise t-tests with Bonferroni correction
from scipy.stats import ttest_ind
from itertools import combinations

group_data = {
    'A': treatment_a,
    'B': treatment_b, 
    'C': treatment_c,
    'Control': control
}

comparisons = list(combinations(group_data.keys(), 2))
p_values_posthoc = []

for group1, group2 in comparisons:
    t_stat, p_val = ttest_ind(group_data[group1], group_data[group2])
    p_values_posthoc.append(p_val)

# Apply Bonferroni correction
corrected_p_posthoc = multipletests(p_values_posthoc, method='bonferroni')[1]
for (group1, group2), p_corr in zip(comparisons, corrected_p_posthoc):
    pass  # Loop body

## 4. Bootstrap Methods

Bootstrap resampling for robust statistical inference and confidence intervals.

In [ ]:
import numpy as np
# Bootstrap confidence intervals for mean
def bootstrap_statistic(data, statistic_func, n_bootstrap=1000, confidence_level=0.95):
    """Calculate bootstrap confidence intervals for a statistic."""
    bootstrap_stats = []
    n = len(data)
    
    for _ in range(n_bootstrap):
        # Resample with replacement
        bootstrap_sample = np.random.choice(data, size=n, replace=True)
        bootstrap_stats.append(statistic_func(bootstrap_sample))
    
    bootstrap_stats = np.array(bootstrap_stats)
    
    # Calculate confidence intervals
    alpha = 1 - confidence_level
    lower_percentile = 100 * (alpha / 2)
    upper_percentile = 100 * (1 - alpha / 2)
    
    ci_lower = np.percentile(bootstrap_stats, lower_percentile)
    ci_upper = np.percentile(bootstrap_stats, upper_percentile)
    
    return {
        'statistic': statistic_func(data),
        'bootstrap_mean': np.mean(bootstrap_stats),
        'bootstrap_std': np.std(bootstrap_stats),
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'bootstrap_distribution': bootstrap_stats
    }

# Bootstrap for mean
sample_data = np.random.gamma(2, 2, 100)  # Skewed distribution
bootstrap_mean = bootstrap_statistic(sample_data, np.mean)


In [ ]:
import numpy as np
# Bootstrap for median (robust statistic)
bootstrap_median = bootstrap_statistic(sample_data, np.median)


In [ ]:
import matplotlib.pyplot as plt
# Visualize bootstrap distributions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Bootstrap distribution of mean
ax1.hist(bootstrap_mean['bootstrap_distribution'], bins=50, alpha=0.7, density=True)
ax1.axvline(bootstrap_mean['statistic'], color='red', linestyle='--', label='Original mean')
ax1.axvline(bootstrap_mean['ci_lower'], color='orange', linestyle='--', label='95% CI')
ax1.axvline(bootstrap_mean['ci_upper'], color='orange', linestyle='--')
ax1.set_xlabel('Bootstrap Mean Values')
ax1.set_ylabel('Density')
ax1.set_title('Bootstrap Distribution of Mean')
ax1.legend()

# Bootstrap distribution of median
ax2.hist(bootstrap_median['bootstrap_distribution'], bins=50, alpha=0.7, density=True)
ax2.axvline(bootstrap_median['statistic'], color='red', linestyle='--', label='Original median')
ax2.axvline(bootstrap_median['ci_lower'], color='orange', linestyle='--', label='95% CI')
ax2.axvline(bootstrap_median['ci_upper'], color='orange', linestyle='--')
ax2.set_xlabel('Bootstrap Median Values')
ax2.set_ylabel('Density')
ax2.set_title('Bootstrap Distribution of Median')
ax2.legend()

plt.tight_layout()
plt.show()

## 5. PyTorch Integration

Statistical operations with PyTorch tensors for GPU acceleration and deep learning integration.

In [ ]:
# Create PyTorch tensors
torch_data1 = torch.randn(1000, requires_grad=True)
torch_data2 = torch.randn(1000, requires_grad=True)


In [ ]:
import numpy as np
# Correlation with PyTorch tensors
def torch_correlation(x, y):
    """Calculate Pearson correlation using PyTorch."""
    mean_x = torch.mean(x)
    mean_y = torch.mean(y)
    
    numerator = torch.sum((x - mean_x) * (y - mean_y))
    denominator = torch.sqrt(torch.sum((x - mean_x)**2) * torch.sum((y - mean_y)**2))
    
    return numerator / denominator

# Calculate correlation
correlation = torch_correlation(torch_data1, torch_data2)

# Compare with numpy correlation
numpy_corr = np.corrcoef(torch_data1.detach().numpy(), torch_data2.detach().numpy())[0, 1]

In [ ]:
# Statistical moments with PyTorch
def torch_moments(x, max_moment=4):
    """Calculate statistical moments using PyTorch."""
    mean = torch.mean(x)
    centered = x - mean
    
    moments = {'mean': mean.item()}
    
    for k in range(2, max_moment + 1):
        moment_k = torch.mean(centered ** k)
        moments[f'moment_{k}'] = moment_k.item()
    
    # Calculate derived statistics
    variance = moments['moment_2']
    std = torch.sqrt(torch.tensor(variance))
    skewness = moments['moment_3'] / (std ** 3)
    kurtosis = moments['moment_4'] / (variance ** 2) - 3  # Excess kurtosis
    
    moments.update({
        'variance': variance,
        'std': (std.item() if hasattr(std, "item") else float(std)) if hasattr(std, "item") else std,
        'skewness': (skewness.item() if hasattr(skewness, "item") else float(skewness)) if hasattr(skewness, "item") else skewness,
        'kurtosis': (kurtosis.item() if hasattr(kurtosis, "item") else float(kurtosis)) if hasattr(kurtosis, "item") else kurtosis
    })
    
    return moments

# Calculate moments
moments = torch_moments(torch_data1)
for key, value in moments.items():
    pass  # Loop body

## 6. Outlier Detection

Multiple methods for detecting and handling outliers in data.

In [ ]:
import numpy as np
# Generate data with outliers
np.random.seed(42)
normal_data = np.random.normal(0, 1, 100)
outliers = np.array([5, -4, 6, -5])  # Extreme values
data_with_outliers = np.concatenate([normal_data, outliers])


In [ ]:
import numpy as np
# Z-score method for outlier detection
def detect_outliers_zscore(data, threshold=3):
    """Detect outliers using Z-score method."""
    z_scores = np.abs((data - np.mean(data)) / np.std(data))
    outlier_indices = np.where(z_scores > threshold)[0]
    return outlier_indices, z_scores

outlier_idx_z, z_scores = detect_outliers_zscore(data_with_outliers)

In [ ]:
import numpy as np
# IQR method for outlier detection
def detect_outliers_iqr(data, factor=1.5):
    """Detect outliers using Interquartile Range method."""
    q1 = np.percentile(data, 25)
    q3 = np.percentile(data, 75)
    iqr = q3 - q1
    
    lower_bound = q1 - factor * iqr
    upper_bound = q3 + factor * iqr
    
    outlier_indices = np.where((data < lower_bound) | (data > upper_bound))[0]
    return outlier_indices, (lower_bound, upper_bound)

outlier_idx_iqr, bounds = detect_outliers_iqr(data_with_outliers)

In [ ]:
import numpy as np
# Modified Z-score (robust) method
def detect_outliers_modified_zscore(data, threshold=3.5):
    """Detect outliers using Modified Z-score method (robust)."""
    median = np.median(data)
    mad = np.median(np.abs(data - median))  # Median Absolute Deviation
    
    modified_z_scores = 0.6745 * (data - median) / mad
    outlier_indices = np.where(np.abs(modified_z_scores) > threshold)[0]
    
    return outlier_indices, modified_z_scores

outlier_idx_mod, mod_z_scores = detect_outliers_modified_zscore(data_with_outliers)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Visualize outlier detection results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Original data
axes[0, 0].hist(data_with_outliers, bins=30, alpha=0.7)
axes[0, 0].axvline(data_with_outliers.mean(), color='red', linestyle='--', label='Mean')
axes[0, 0].axvline(np.median(data_with_outliers), color='green', linestyle='--', label='Median')
axes[0, 0].set_title('Original Data with Outliers')
axes[0, 0].legend()

# Z-score outliers
axes[0, 1].scatter(range(len(data_with_outliers)), data_with_outliers, alpha=0.6)
axes[0, 1].scatter(outlier_idx_z, data_with_outliers[outlier_idx_z], color='red', s=50, label='Z-score outliers')
axes[0, 1].set_title('Z-score Outlier Detection')
axes[0, 1].set_xlabel('Index')
axes[0, 1].set_ylabel('Value')
axes[0, 1].legend()

# IQR outliers
axes[1, 0].scatter(range(len(data_with_outliers)), data_with_outliers, alpha=0.6)
axes[1, 0].scatter(outlier_idx_iqr, data_with_outliers[outlier_idx_iqr], color='orange', s=50, label='IQR outliers')
axes[1, 0].axhline(bounds[0], color='red', linestyle='--', alpha=0.5, label='IQR bounds')
axes[1, 0].axhline(bounds[1], color='red', linestyle='--', alpha=0.5)
axes[1, 0].set_title('IQR Outlier Detection')
axes[1, 0].set_xlabel('Index')
axes[1, 0].set_ylabel('Value')
axes[1, 0].legend()

# Modified Z-score outliers
axes[1, 1].scatter(range(len(data_with_outliers)), data_with_outliers, alpha=0.6)
axes[1, 1].scatter(outlier_idx_mod, data_with_outliers[outlier_idx_mod], color='purple', s=50, label='Modified Z-score outliers')
axes[1, 1].set_title('Modified Z-score Outlier Detection')
axes[1, 1].set_xlabel('Index')
axes[1, 1].set_ylabel('Value')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

## 7. Power Analysis

Statistical power analysis for experimental design and sample size determination.

In [ ]:
import numpy as np
# Power analysis for t-test
from scipy import stats

def power_ttest(effect_size, n, alpha=0.05, alternative='two-sided'):
    """Calculate power for two-sample t-test."""
    # Calculate degrees of freedom
    df = 2 * n - 2
    
    # Critical t-value
    if alternative == 'two-sided':
        t_crit = stats.t.ppf(1 - alpha/2, df)
    else:
        t_crit = stats.t.ppf(1 - alpha, df)
    
    # Non-centrality parameter
    ncp = effect_size * np.sqrt(n/2)
    
    # Power calculation
    if alternative == 'two-sided':
        power = 1 - stats.nct.cdf(t_crit, df, ncp) + stats.nct.cdf(-t_crit, df, ncp)
    else:
        power = 1 - stats.nct.cdf(t_crit, df, ncp)
    
    return power

# Example: power analysis for different effect sizes
effect_sizes = [0.2, 0.5, 0.8]  # Small, medium, large
sample_size = 30

for es in effect_sizes:
    power = power_ttest(es, sample_size)

In [ ]:
# Sample size calculation for desired power
def sample_size_ttest(effect_size, power=0.8, alpha=0.05):
    """Calculate required sample size for desired power."""
    # Use iterative approach to find sample size
    n = 10
    while n < 1000:
        calculated_power = power_ttest(effect_size, n, alpha)
        if calculated_power >= power:
            return n
        n += 1
    return n

for es in effect_sizes:
    n_required = sample_size_ttest(es)

In [ ]:
import matplotlib.pyplot as plt
# Power curves
sample_sizes = range(5, 101, 5)
power_curves = {}

for es in effect_sizes:
    powers = [power_ttest(es, n) for n in sample_sizes]
    power_curves[es] = powers

# Plot power curves
plt.figure(figsize=(10, 6))
for es in effect_sizes:
    plt.plot(sample_sizes, power_curves[es], label=f'Effect size = {es}', linewidth=2)

plt.axhline(y=0.8, color='red', linestyle='--', alpha=0.7, label='80% Power')
plt.xlabel('Sample Size (per group)')
plt.ylabel('Statistical Power')
plt.title('Power Curves for Two-Sample t-test')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 1)
plt.show()

## 8. Advanced Statistical Operations

Complex statistical operations including permutation tests and robust estimators.

In [ ]:
import numpy as np
# Permutation test for difference in means
def permutation_test(group1, group2, n_permutations=10000, statistic_func=None):
    """Perform permutation test for difference between groups."""
    if statistic_func is None:
        statistic_func = lambda x, y: np.mean(x) - np.mean(y)
    
    # Observed test statistic
    observed_stat = statistic_func(group1, group2)
    
    # Combine groups
    combined = np.concatenate([group1, group2])
    n1, n2 = len(group1), len(group2)
    
    # Permutation distribution
    perm_stats = []
    for _ in range(n_permutations):
        # Randomly permute the combined data
        permuted = np.random.permutation(combined)
        perm_group1 = permuted[:n1]
        perm_group2 = permuted[n1:]
        
        perm_stat = statistic_func(perm_group1, perm_group2)
        perm_stats.append(perm_stat)
    
    perm_stats = np.array(perm_stats)
    
    # Two-tailed p-value
    p_value = np.mean(np.abs(perm_stats) >= np.abs(observed_stat))
    
    return {
        'observed_statistic': observed_stat,
        'p_value': p_value,
        'permutation_distribution': perm_stats
    }

# Example permutation test
group_a = np.random.normal(5, 2, 50)
group_b = np.random.normal(6, 2, 50)

perm_result = permutation_test(group_a, group_b)

# Compare with parametric t-test
t_stat, t_p = stats.ttest_ind(group_a, group_b)

In [ ]:
import numpy as np
# Robust statistics
def robust_statistics(data):
    """Calculate robust statistical measures."""
    # Central tendency
    mean = np.mean(data)
    median = np.median(data)
    trimmed_mean = stats.trim_mean(data, 0.1)  # 10% trimmed mean
    
    # Variability
    std = np.std(data)
    mad = np.median(np.abs(data - median))  # Median Absolute Deviation
    iqr = np.percentile(data, 75) - np.percentile(data, 25)
    
    return {
    'mean': mean,
    'median': median,
    'trimmed_mean': trimmed_mean,
    'std': std,
    'mad': mad,
    'iqr': iqr
    }

# Compare robust vs non-robust statistics
clean_data = np.random.normal(0, 1, 100)
contaminated_data = np.concatenate([clean_data, [10, -8, 12]])  # Add outliers

clean_stats = robust_statistics(clean_data)
for key, value in clean_stats.items():
        pass  # Processing key
contam_stats = robust_statistics(contaminated_data)
for key, value in contam_stats.items():
        pass  # Processing key
for key in clean_stats.keys():
    pct_change = 100 * abs(contam_stats[key] - clean_stats[key]) / abs(clean_stats[key])

## 9. Complete Statistical Analysis Pipeline

A comprehensive example combining multiple statistical techniques for a complete analysis.

In [ ]:
import pandas as pd
import numpy as np
# Generate realistic experimental data
np.random.seed(42)

# Simulate a psychological experiment with 4 conditions
n_per_group = 25
conditions = ['Control', 'Treatment_A', 'Treatment_B', 'Treatment_C']
true_effects = [0, 0.5, 1.0, 0.3]  # Effect sizes

experimental_data = []
for i, (condition, effect) in enumerate(zip(conditions, true_effects)):
    # Add some realistic noise and individual differences
    scores = np.random.normal(100 + effect * 10, 15, n_per_group)
    
    # Add a few outliers to make it realistic
    if np.random.random() < 0.1:  # 10% chance of outlier
    outlier_idx = np.random.randint(0, n_per_group)
    scores[outlier_idx] += np.random.choice([-1, 1]) * np.random.uniform(30, 50)
    
    for score in scores:
        experimental_data.append({
        'condition': condition,
        'score': score,
        'subject_id': f"{condition}_{len(experimental_data) % n_per_group + 1}"
        })

# Create DataFrame
df_exp = pd.DataFrame(experimental_data)


In [ ]:
# Step 1: Exploratory Data Analysis

# Check for outliers in each group
for condition in conditions:
    group_data = df_exp[df_exp['condition'] == condition]['score'].values
    outliers_z, _ = detect_outliers_zscore(group_data, threshold=2.5)
    outliers_iqr, _ = detect_outliers_iqr(group_data)
    
    if len(outliers_z) > 0:
        pass  # Condition handled

# Normality tests
for condition in conditions:
    group_data = df_exp[df_exp['condition'] == condition]['score'].values
    stat, p = stats.shapiro(group_data)

In [ ]:
import numpy as np
# Step 2: Main Analysis - ANOVA

# Prepare data for ANOVA
group_arrays = [df_exp[df_exp['condition'] == cond]['score'].values for cond in conditions]

# One-way ANOVA
f_stat, p_anova = stats.f_oneway(*group_arrays)

# Effect size (eta-squared)
ss_between = sum([len(group) * (np.mean(group) - np.mean(df_exp['score']))**2 for group in group_arrays])
ss_total = sum([(score - np.mean(df_exp['score']))**2 for score in df_exp['score']])
eta_squared = ss_between / ss_total

# Interpretation
if eta_squared < 0.01:
    effect_size_interp = "small"
elif eta_squared < 0.06:
    effect_size_interp = "medium"
else:
    effect_size_interp = "large"

In [ ]:
import numpy as np
# Step 3: Post-hoc Analysis

if p_anova < 0.05:
    
    # Pairwise comparisons with multiple correction
    from itertools import combinations
    
    pairwise_results = []
    p_values_posthoc = []
    
    for i, j in combinations(range(len(conditions)), 2):
        group1_data = group_arrays[i]
        group2_data = group_arrays[j]
        
        # t-test
        t_stat, p_val = stats.ttest_ind(group1_data, group2_data)
        
        # Effect size (Cohen's d)
        pooled_std = np.sqrt(((len(group1_data)-1)*np.var(group1_data, ddof=1) + 
        (len(group2_data)-1)*np.var(group2_data, ddof=1)) /
        (len(group1_data) + len(group2_data) - 2))
        cohens_d = (np.mean(group1_data) - np.mean(group2_data)) / pooled_std
        
        pairwise_results.append({
        'comparison': f"{conditions[i]} vs {conditions[j]}",
        't_stat': t_stat,
        'p_value': p_val,
        'cohens_d': cohens_d
        })
        p_values_posthoc.append(p_val)
    
    # Multiple comparison correction
    corrected_p = multipletests(p_values_posthoc, method='bonferroni')[1]
    
    for result, p_corr in zip(pairwise_results, corrected_p):
        pass  # Processing result
else:    pass  # Fixed incomplete block


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# Step 4: Visualization

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Box plot
df_exp.boxplot(column='score', by='condition', ax=axes[0, 0])
axes[0, 0].set_title('Distribution by Condition')
axes[0, 0].set_xlabel('Condition')
axes[0, 0].set_ylabel('Score')

# Violin plot (using matplotlib)
group_data_for_violin = [df_exp[df_exp['condition'] == cond]['score'].values for cond in conditions]
violin_parts = axes[0, 1].violinplot(group_data_for_violin, positions=range(1, len(conditions)+1))
axes[0, 1].set_xticks(range(1, len(conditions)+1))
axes[0, 1].set_xticklabels(conditions, rotation=45)
axes[0, 1].set_title('Density Distribution by Condition')
axes[0, 1].set_ylabel('Score')

# Mean with error bars
means = [np.mean(group) for group in group_data_for_violin]
sems = [stats.sem(group) for group in group_data_for_violin]
axes[1, 0].bar(conditions, means, yerr=sems, capsize=5, alpha=0.7)
axes[1, 0].set_title('Means with Standard Error')
axes[1, 0].set_ylabel('Score')
axes[1, 0].tick_params(axis='x', rotation=45)

# Q-Q plots for normality check
for i, (condition, group_data) in enumerate(zip(conditions, group_data_for_violin)):
    if i < 2:  # Only show first 2 for space
    continue
    stats.probplot(group_data, dist="norm", plot=axes[1, 1])
    break
axes[1, 1].set_title('Q-Q Plot (Normality Check)')

plt.tight_layout()
plt.show()

if p_anova < 0.05:
    significant_pairs = [result['comparison'] for result, p_corr in zip(pairwise_results, corrected_p) if p_corr < 0.05]
    for pair in significant_pairs:
        # Process pair

## Summary

This comprehensive tutorial has demonstrated the full capabilities of the SciTeX statistics module:

### Key Features Covered:
1. **Basic Statistical Tests** - t-tests, ANOVA, chi-square tests with automatic result formatting
2. **Correlation Analysis** - Pearson and Spearman correlations with multiple comparison corrections
3. **Advanced Methods** - Bootstrap resampling, permutation tests, and robust statistics
4. **PyTorch Integration** - Statistical operations on tensors for GPU acceleration
5. **Outlier Detection** - Multiple methods (Z-score, IQR, Modified Z-score) for identifying anomalies
6. **Power Analysis** - Sample size calculation and power curves for experimental design
7. **Complete Workflows** - End-to-end statistical analysis pipelines

### Best Practices Demonstrated:
- Always check assumptions (normality, homogeneity of variance)
- Use appropriate multiple comparison corrections
- Report effect sizes alongside significance tests
- Consider robust alternatives when data contains outliers
- Visualize data to understand patterns and validate assumptions
- Use bootstrap and permutation methods for non-parametric inference

The SciTeX stats module provides a comprehensive toolkit for statistical analysis in scientific computing, combining ease of use with statistical rigor.